In [1]:
from normalorder.sense.bifurcation_finder import *
from tqdm import tqdm
import pickle
from multiprocessing import Pool

In [2]:
def parallel_wrapper(params):
    idx = params[0]
    try:
        bf = task(*params[1:])
        results = extract_results(bf)
        success = 1
    except:
        results = None
        success = 0
    finally:
        pickle.dump(results, open("./output/{}.p".format(idx), "wb" ))
        return success

In [3]:
Z = 50 # ohms
v_p = 1.1817e8 # m/s
l = 0.007809801198766881 # m
L_0 = Z / v_p # H/m
C_0 = 1 /(v_p*Z) # F/m
x_J = 0.0 # m
Q_a = 10000

phi_ext_initial = [-0.2]

collected_signals = []

#f_J_array = np.linspace(0.3, 3.0,11)*1e11
phi_squid = 0.5
threshold = 0.5

f_J_snail_array = np.linspace(300, 3000, 10)*1e9
f_J_squid_array = np.linspace(300, 3000, 10)*1e9
f_J_snail_grid, f_J_squid_array = np.meshgrid(f_J_snail_array, f_J_squid_array)
parameters_array = np.vstack([f_J_snail_grid.flatten(), f_J_squid_array.flatten()]).T

collected_params = []

for idx in tqdm(range(parameters_array.shape[0])):
    f_J_snail, f_J_squid = parameters_array[idx]
    f_J_1 = f_J_squid
    f_J_2 = 1.04*f_J_squid
    Q = 10000
    resonator_params = {'x_J': x_J, 'L_0': L_0, 'l': l, 'C_0': C_0}
    potential_params = {'f_J': f_J_snail, 'f_J_1': f_J_1, 'f_J_2': f_J_2, 'phi_squid': phi_squid}
    
    params = [idx, potential_params, resonator_params, Q, phi_squid]
    collected_params.append(params)

100%|█████████████████████████████████| 25600/25600 [00:00<00:00, 181652.08it/s]


In [ ]:
pool = Pool(processes=2)
success = pool.map(parallel_wrapper, collected_params)